In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import os
import json
import re
import time
import datetime
from pathlib import Path

In [37]:
def adicionar_categoria(categoria, diretorio, expressao):
    if os.path.isfile('ControleArquivos.json'):
        with open('ControleArquivos.json', 'r') as controle:
            data = json.load(controle)
        if categoria not in data:
            data[categoria] = {
                "diretorio": diretorio,
                "expressao": expressao,
                "arquivos": []
            }
            if not os.path.isdir(data[categoria]["diretorio"]):
                os.makedirs(data[categoria]["diretorio"])

        with open('ControleArquivos.json', 'w') as controle:
            json.dump(data, controle, indent=4)

def retornar_dados():
    if os.path.isfile('ControleArquivos.json') == False:
        return {}

    with open('ControleArquivos.json', 'r') as controle:
        data = json.load(controle)
    return data

def retornar_meses_pendentes(categoria, ano):
    meses = range(1, 13)
    dados = retornar_dados()
    meses_inseridos = set(int(item['mes']) for item in dados[categoria]['arquivos'] if item['ano'] == str(ano))
    if ano == datetime.datetime.now().year:
        meses = range(1, datetime.datetime.now().month + 1)

    meses_pendentes = [mes for mes in meses if mes not in meses_inseridos]
    return meses_pendentes

def retornar_arquivos_pendentes(categorias):
    pendentes = {}
    for categoria in categorias:
        adicionar_categoria(
            categoria,
            categorias[categoria]['diretorio'] if 'diretorio' in categorias[categoria] else os.path.join(os.getcwd(), os.path.abspath('..\\'), 'Bases', categoria),
            categorias[categoria]['expressao']
        )
        pendentes[categoria] = {}
        for ano in anos:
            meses = retornar_meses_pendentes(categoria, ano)
            if meses:
                pendentes[categoria][ano] = {'meses': meses}

        if not pendentes[categoria]:
            pendentes.pop(categoria)
    return pendentes

def adicionar_arquivo(categoria, arquivo, ano, mes, tamanho, data):
    dados = retornar_dados()
    dados[categoria]['arquivos'].append({
        'arquivo': str(arquivo),
        'ano': str(ano),
        'mes': str(mes),
        'tamanho': str(tamanho),
        'data': str(data)
    })
    with open('ControleArquivos.json', 'w') as controle:
        json.dump(dados, controle, indent=4)



In [38]:
botoes = {
    'categoria': 'cphBody_btn',
    'ano': 'cphBody_lkAno',
    'mes': 'cphBody_lkMes',
    'exportar': 'cphBody_ExportarBOLink'
}

def abrir_navegador(link):
    opcoes = Options()
    # options.add_argument('--headless')
    navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opcoes)
    navegador.get(link)
    return navegador

def clicar_botao(botao, valor):
    elemento = (By.XPATH, f'//*[@id="{botoes[botao]}{valor}"]')
    antes = datetime.datetime.now()
    print(f'ANTES  | {botao} | {valor} | {antes}')

    while True:
        try:
            WebDriverWait(navegador, 10).until(EC.element_to_be_clickable(elemento)).click()
            print(f'DEPOIS | {botao} | {valor} | {datetime.datetime.now() - antes} - FINALIZADO')
            # Verifique aqui se o download foi bem-sucedido
            break
        except TimeoutException:
            print("TimeoutException: O botão não está clicável ainda. Aguardando...")
            time.sleep(5)


In [39]:
categorias = {
    'FurtoCelular': {
        'expressao': r'DadosBO_(\d{4})_(\d{1,2})\(FURTO DE CELULAR\)\.(\w+)',
    },
    'RouboCelular': {
        'expressao': r'DadosBO_(\d{4})_(\d{1,2})\(ROUBO DE CELULAR\)\.(\w+)'
    }
}

anos = range(2010, datetime.datetime.now().year + 1)

In [43]:
# arquivos_pendentes = retornar_arquivos_pendentes(categorias)
arquivos_pendentes = {
    'FurtoCelular': {
         2012: {
             'meses': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
             }}}

if arquivos_pendentes == {}:
    print('Não há arquivos pendentes para baixar')
    exit()

link = 'http://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx'
navegador = abrir_navegador(link)
for categoria in arquivos_pendentes:
    clicar_botao('categoria', categoria)
    for ano in arquivos_pendentes[categoria]:
        clicar_botao('ano', ano % 100)
        for mes in arquivos_pendentes[categoria][ano]['meses']:
            clicar_botao('mes', mes)
            time.sleep(20)
            clicar_botao('exportar', '')

            downloads = str(Path.home() / "Downloads")
            regex_pattern = r'DadosBO_(\d{4})_(\d{1,2})\(FURTO DE CELULAR\)\.(\w+)'

            matching_files = []
            while matching_files == []:
                try:
                    files = os.listdir(downloads)
                    matching_files = [filename for filename in files if re.match(regex_pattern, filename)]
                    print(matching_files)
                    matching_files = [item for item in matching_files if '.crdownload' not in item]
                    print(f'Arquivos encontrados: {matching_files}')

                    if matching_files:
                        matching_files.sort(reverse=True)
                        ultimo_arquivo_baixado = matching_files[0]
                        adicionar_arquivo(categoria,
                                        ultimo_arquivo_baixado,
                                        ano,
                                        mes,
                                        f'{os.path.getsize(os.path.join(downloads, ultimo_arquivo_baixado)) / 1024:.2f} KB',
                                        datetime.datetime.fromtimestamp(os.path.getmtime(ultimo_arquivo_baixado)))
                        os.rename(os.path.join(downloads, ultimo_arquivo_baixado),
                                r'c:\\TCC\\Bases\\FurtoCelular')
                        break
                except FileNotFoundError:
                    print('Arquivo não encontrado, aguardando...')
                    time.sleep(5)



ANTES  | categoria | FurtoCelular | 2023-10-31 22:17:43.779996
DEPOIS | categoria | FurtoCelular | 0:00:00.069694 - FINALIZADO
ANTES  | ano | 12 | 2023-10-31 22:17:43.850686
DEPOIS | ano | 12 | 0:00:01.658343 - FINALIZADO
ANTES  | mes | 3 | 2023-10-31 22:17:45.509029
DEPOIS | mes | 3 | 0:00:01.137188 - FINALIZADO
ANTES  | exportar |  | 2023-10-31 22:18:06.646926
DEPOIS | exportar |  | 0:03:25.041869 - FINALIZADO
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['DadosBO_2012_3(FURTO DE CELULAR).xls.crdownload']
Arquivo não encontrado, aguardando...
ANTES  | mes | 4 | 2023-10-31 22:21:36.741006
DEPOIS | mes | 4 | 0:00:20.503578 - FINALIZADO
ANTES  | exportar |  | 2023-10-31 22:22:17.258135
DEPOIS | exportar |  | 0:03:14.689338 - FINALIZADO
['DadosBO_2012_3(FURTO DE CELULAR).xls']
Arquivo não encontrado, aguardando...
ANTES  | mes | 5 | 2023-10-31 22:25:36.95698

NameError: name 'TimeoutException' is not defined

In [47]:
files = os.listdir(downloads)
matching_files = [filename for filename in files if re.match(regex_pattern, filename)]
matching_files.append('DadosBO_2021_1(FURTO DE CELULAR).xls.crdownload')
print(matching_files)
matching_files = [item for item in matching_files if '.crdownload' not in item]
print(f'Arquivos encontrados: {matching_files}')

['DadosBO_2010_2(FURTO DE CELULAR).xls', 'DadosBO_2021_1(FURTO DE CELULAR).xls.crdownload']
Arquivos encontrados: ['DadosBO_2010_2(FURTO DE CELULAR).xls']


In [42]:
downloads = str(Path.home() / "Downloads")
regex_pattern = r'DadosBO_(\d{4})_(\d{1,2})\(FURTO DE CELULAR\)\.(\w+)'

try:
    files = os.listdir(downloads)
    matching_files = [filename for filename in files if re.match(regex_pattern, filename)]
    print(matching_files)

    if matching_files:
        matching_files.sort(reverse=True)
        ultimo_arquivo_baixado = matching_files[0]
        adicionar_arquivo(categoria,
                        ultimo_arquivo_baixado,
                        ano,
                        mes,
                        f'{os.path.getsize(os.path.join(downloads, ultimo_arquivo_baixado)) / 1024:.2f} KB',
                        datetime.datetime.fromtimestamp(os.path.getmtime(ultimo_arquivo_baixado)))
        os.rename(os.path.join(downloads, ultimo_arquivo_baixado),
                r'c:\\TCC\\Bases\\FurtoCelular')
except FileNotFoundError:
    print('Arquivo não encontrado, aguardando...')
    time.sleep(5)

[]


In [148]:
categorias = [
    'FurtoCelular',
    'RouboCelular'
]

anos = range(2011, 2012)
arquivos_pendentes = retornar_arquivos_pendentes()

navegador = abrir_navegador()
for categoria in arquivos_pendentes:
    clicar_botao('categoria', categoria)
    for ano in arquivos_pendentes[categoria]:
        clicar_botao('ano', ano % 100)
        for mes in arquivos_pendentes[categoria][ano]['meses']:
            clicar_botao('mes', mes)
            time.sleep(20)
            clicar_botao('exportar', '')
            time.sleep(10)


ANTES: categoria | FurtoCelular | 2023-10-30 00:12:19.681758
DEPOIS: categoria | FurtoCelular | 2023-10-30 00:12:35.186332
ANTES: ano | 11 | 2023-10-30 00:12:35.186332
DEPOIS: ano | 11 | 2023-10-30 00:12:42.831664
ANTES: mes | 1 | 2023-10-30 00:12:42.832659
DEPOIS: mes | 1 | 2023-10-30 00:12:45.968078
ANTES: exportar |  | 2023-10-30 00:12:45.968078
DEPOIS: exportar |  | 2023-10-30 00:15:05.993584
ANTES: mes | 2 | 2023-10-30 00:15:05.993584
DEPOIS: mes | 2 | 2023-10-30 00:15:27.706631
ANTES: exportar |  | 2023-10-30 00:15:27.706631
DEPOIS: exportar |  | 2023-10-30 00:17:38.796241
ANTES: mes | 3 | 2023-10-30 00:17:38.796241


ElementClickInterceptedException: Message: element click intercepted: Element <a id="cphBody_lkMes3" class="block" href="javascript:__doPostBack('ctl00$cphBody$lkMes3','')">...</a> is not clickable at point (233, 20). Other element would receive the click: <div class="blockUI blockOverlay" style="z-index: 1000; border: none; margin: 0px; padding: 0px; width: 100%; height: 100%; top: 0px; left: 0px; background-color: rgb(0, 0, 0); opacity: 0.525033; cursor: default; position: fixed;"></div>
  (Session info: chrome=118.0.5993.118)
Stacktrace:
	GetHandleVerifier [0x011A4DE3+43907]
	(No symbol) [0x01130741]
	(No symbol) [0x010233ED]
	(No symbol) [0x0105B5B1]
	(No symbol) [0x0105A0AF]
	(No symbol) [0x0105869B]
	(No symbol) [0x01057A35]
	(No symbol) [0x0104FF9A]
	(No symbol) [0x01072B5C]
	(No symbol) [0x0104F9D6]
	(No symbol) [0x01072DD4]
	(No symbol) [0x010855CA]
	(No symbol) [0x01072956]
	(No symbol) [0x0104E17E]
	(No symbol) [0x0104F32D]
	GetHandleVerifier [0x01455AF9+2865305]
	GetHandleV